## K-Means
K-means é um algoritmo de clustering muito popular, porque é simples e fácil de implementar e mostrou bom desempenho em diferentes tarefas. Pertence à classe de **algoritmos de partição** que particionam pontos de dados simultaneamente em grupos distintos chamados clusters. 

O algoritmo k-means pertence à categoria de agrupamento baseado em protótipo. Agrupamento baseado em protótipo significa que cada agrupamento é representado por um protótipo, que pode ser o centroide (média) de pontos semelhantes com recursos contínuos ou o medoid (o ponto mais representativo ou mais frequente) no caso de recursos categóricos.

A idéia principal por trás do algoritmo é encontrar uma partição de pontos de dados de forma que a distância ao quadrado entre a média do cluster e cada ponto no cluster seja minimizada. O objetivo é **minimizar as diferenças dentro de cada cluster** e **maximizar a diferença entre os clusters**. Observe que esse método pressupõe que você saiba a priori o número de clusters nos quais seus dados devem ser divididos.  

O processamento dos dados utilizado no método k-means é muito próximo ao utilizado pelo KNN. O K-means utiliza distância, portanto os **dados devem ser sempre numéricos**.  

**Expectation–Maximization**  
A menos que os valores de K e n sejam extremamente baixos, não é possível calcular o clusters ótimos para todas as combinações possíveis de exemplos, pois o número de combinações possíveis é exponencial ao número de pontos de dados - uma pesquisa exaustiva seria muito, muito custosa. Em vez disso, o algoritmo utiliza um processo heurístico, que encontra soluções localmente ótimas; a abordagem típica de k-means envolve uma abordagem iterativa intuitiva conhecida como **Expectation–Maximization**.

**Heurística**  
Em resumo, a abordagem Expectation–Maximization (E-M) consiste no seguinte procedimento: 

    1 - O algoritmo atribui aleatoriamente as centróides;
    2 - Repete até convergir:
        a. E-Step: atribui pontos à centróide mais próxima (assign points)
        b. M-Step: desloca a centróide para a média dos pontos do cluster (recompute centers)
    O algoritmo é finalizado quando a atribuição de instâncias aos clusters não for mais alterada ou o número máximo de iterações for alcançado.

<img src="figuras/k-means.png" width="600"/>

Aqui, o “E-step” ou “Etapa de Expectativa” é assim chamado porque envolve a atualização de nossa expectativa de qual cluster cada ponto pertence. A "etapa M" ou "Etapa de maximização" é assim denominada porque envolve a maximização de algumas funções de adequação que definem a localização dos centros de cluster - nesse caso, a maximização é obtida tomando uma média simples dos dados em cada cluster.

A literatura sobre este algoritmo é vasta, mas pode ser resumida da seguinte forma: em circunstâncias típicas, cada repetição do E-step e do M-step sempre resultará em uma melhor estimativa das características do cluster.

**K-Means ++**  
Em 2007, foi introduzido um algoritmo chamado k-means++, que propõe um método alternativo para a seleção dos centros dos cluster iniciais, pois esse é um ponto crítico quando se utiliza K-Means.  
Em vez de utilizar centróides de maneira aleatória, esse algoritmo busca definir os clusters utilizando o **método de Voronoi**, para em seguida, buscar o seu centro através da distância euclidiana entre os pontos. Em seguida esse método é repetido até que o ajuste desses itens não surtam mais efeitos na homogeneidade dos itens que compões cada um dos clusters. 

Para usar k-means ++ no scikit-learn, precisamos apenas definir o parâmetro ``init`` como k-means ++ (que é o padrão) em vez de random (aleatório).

**Alguns cuidados com k-means**

*Ótimo local*  
Às vezes, devido à aleatoriedade das centróideis iniciais, a configuração inicial das posições pode ser azarada e embora seja garantido que o procedimento E-M melhore o resultado em cada etapa, não há garantia de que ele levará à melhor solução global, gerando uma convergência pobre de clusters (ótimo local). Por esta razão, é comum que o algoritmo seja executado em várias tentativas iniciais. Por padrão, o scikit-learn executa o algoritmo 10 vezes com 10 diferentes inicializações aleatórias, e retorna o melhor resultado (melhor aqui significando que a soma das variâncias dos clusters é pequena). 

*Característica dos dados*  
Devido ao modo como calcula as distâncias entre os pontos, o k-means fica limitado a clusters com limites lineares, ou seja, o algoritmo geralmente será ineficaz se os clusters tiverem geometrias complicadas. Para esses casos necessitamos utilizar o **kernel trick** que adicionar mais de uma dimensão aos dados, permitindo utilizar uma linha reta que separa os conjuntos. Uma versão do k-means kernelizado foi implementada no Scikit-Learn dentro do estimador **SpectralClustering**. Ele usa a metodologia do grafo do vizinho mais próximo (graph of nearest neighbors) para calcular uma representação dimensional mais alta dos dados e, em seguida, atribui rótulos usando o algoritmo k-means. Há um bom exemplo dessa utilização no próximo notebook.

*Escala*  
Quando aplicamos k-means em dados do mundo real usando a métrica de distância euclidiana, queremos ter certeza de que os recursos serão medidos na mesma escala, portanto se necessário, devemos aplicar a padronização z-score ou o escalonamento min-max.

*Lentidão para um grande número de amostras*  
Como cada iteração de k-means deve acessar todos os pontos no conjunto de dados, o algoritmo pode ser relativamente lento à medida que o número de amostras aumenta. Você pode se perguntar se esse requisito de usar todos os dados em cada iteração pode ser relaxado; por exemplo, você pode usar apenas um subconjunto dos dados para atualizar os centros de cluster em cada etapa. Essa é a ideia por trás dos algoritmos k-means baseados em batch (lote), sendo uma forma implementada no ``sklearn.cluster.MiniBatchKMeans``. Sua interface é a mesma do KMeans padrão.

*Número de clusters deve ser definido a priori*  
Outro desafio comum com k-means é que devemos informar a quantidade clusters que o algoritmo tem que trabalhar e na maioria das vezes essa não é uma tarefa fácil. Um dos métodos utilizados é o Elbow, detalhado a seguir.

### Escolhendo o número apropriado de clusters
Às vezes, o número de clusters é especificado pelo contexto do problema. Por exemplo, uma empresa que fabrica sapatos sabe que é capaz de suportar a fabricação de três novos modelos. Para entender quais grupos de clientes segmentar com cada modelo, ela pesquisa os clientes e cria três clusters a partir dos resultados. Ou seja, o valor de K foi especificado pelo contexto do problema. Outros problemas podem não exigir um número específico de clusters e o número ideal de clusters pode ser ambíguo.

O K-means é sensível ao número de clusters. Definir um valor de k muito grande irá melhorar a homogeneidade dos clusters, porém corre-se o risco de sobrecarregar os dados. O ideal é que se tenha a priori o conhecimento sobre os verdadeiros agrupamentos para aplicar essa informação ao número de clusters. 

Sem qualquer conhecimento prévio, uma regra geral sugere que k seja igual à raiz quadrada de (n / 2), onde n é o número de exemplos no conjunto de dados. No entando, para um grande conjunto de dados, esse regra pode resultar em um alto número de clusters. Existem alguns outros métodos estatísticos que ajudam a encontrar o valor de K.

**Método de Elbow**  
O método elbow plota o valor da cost function produzida por diferentes valores de K. À medida que K aumenta, a distorção média diminui; cada cluster terá menos instâncias constituintes e as instâncias estarão mais próximas de seus respectivos centróides. No entanto, as melhorias na distorção média diminuirão à medida que K aumenta. O valor de K no qual a melhoria da distorção diminui mais é chamado de cotovelo (elbow). 

<img src="figuras/elbow.png" width="600"/>

Interpretando o gráfico abaixo, à direita: Temos no eixo x a quantidade de centróides, e no eixo y o valor da ``inertia_`` que é um atributo gerado pelo algoritmo, após o treino, que mede a distância dos pontos em relação às centróides atribuídas. Quanto menor a inertia, melhor a classificação feita pelo algoritmo. A inertia também é conhecida como distorção média.
<img src="figuras/elbow2.png" width="600"/>

Na nossa massa de dados há um valor de K que se aproxima da melhor escolha. A partir desse ponto, aumentar seu valor pode piorar o resultado do algoritmo, pois podemos acabar gerando sobrecarga dos dados, ou seja, poucos dados em muitos clusters. Veja no exemplo acima à esquerda que, a partir do k=4, à medida que aumentamos a quantidade de k não há ganho de homogeneidade e temos perda na classificação de todos os dados em cada grupo (completude). No gráfico à direita, podemos ver que o valor de dispersão dos dados em relação à centróide (inertia) apresenta ganhos pequenos a partir de K=4.  
Tanto a homogeneidade quanto a completude são medidos dentro do Cluster, ou seja, mede-se a semelhança entre os pontos de dados dentro de cada cluster a fim de verificar o nível de similaridade e com isso emitir uma métrica que seja capaz de avaliar a eficácia do algoritmo. Não há comparação com dados rotulados, pois eles não existem, uma vez que estamos trabalhando com aprendizagem não supervisionada.

Entretanto, é preciso ter algum critério de comparação, para que as métricas possam ser avaliadas. Para isso, usamos uma amostragem aleatória isotrópica, criando clusters de maneira aleatória e criando um conjunto de dados y. Comparamos então nossas previsões em y_pred (feitas pelo nosso algoritmo K-means) e medimos a métrica. Essa solução foi proposta por Rosenberg and Hirschberg (2007) e dela surgiram as métricas homogeneity e completeness.

Aqui tem mais detalhes sobre as métricas:
http://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure


**Silhouette coefficient**  
Outra métrica intrínseca para avaliar a qualidade de um cluster é a análise de silhueta, que também pode ser aplicada a algoritmos de clustering diferentes de k-means. A análise de silhueta pode ser usada como uma ferramenta gráfica para plotar uma medida de quão fortemente agrupadas são as amostras nos clusters. Para calcular o coeficiente de silhueta de uma única amostra em nosso conjunto de dados, podemos aplicar as três etapas a seguir:

1. Calcule a **coesão** do cluster "a" como a distância média entre uma amostra "x" e todos os outros pontos no mesmo cluster.

2. Calcular a **separação** de cluster "b" do próximo cluster mais próximo da distância média entre a amostra "x" e todas as amostras no cluster mais próximo.

3. Calcule a silhueta "s" como a diferença entre a coesão do cluster e a separação dividida pela maior das duas, como mostrado aqui:

$$s = \frac{b-a}{max(b,a)}$$

*a* é a distância média entre uma amostra e todos os outros pontos desse cluster e *b* é a distância média da mesma amostra até os próximos pontos de cluster mais próximos.

<img src="figuras/silhouette_coefficient.png" width="700"/>



O coeficiente de silhueta é limitado ao intervalo de -1 (ruim) a 1 (bom); Aumenta à medida que a qualidade dos clusters aumenta; é grande para clusters compactos e distantes um do outro; e pequenos para clusters grandes e sobrepostos. 

Analisando a fórmula anterior, podemos ver que o coeficiente de silhueta é 0 se a separação e a coesão do cluster forem iguais (b=a). Além disso, chegamos perto de um coeficiente de silhueta ideal de 1 se b >> a, já que "b" quantifica o quão diferente uma amostra é dos outros clusters, e "a" nos diz como é semelhante às outras amostras em seu próprio cluster, respectivamente.

No exemplo abaixo, foi criado um dataset com 4 conjuntos bem definidos e em seguida feita uma análise de silhueta utilizando 2, 4 e 6 clusters. Perceba que há uma grande assimetria para as silhuetas com 2 e 6 clusters, indicando que a quantidade de clusters que mais representa o conjunto de dados é realmente 4. No eixo y temos as amostras dos clusters e no eixo x o coeficiente calculado com a equação acima. A linha em vermelho é a média dos coeficientes.

<img src="figuras/silhouette_analysis02.png" width="450"/>
<img src="figuras/silhouette_analysis04.png" width="450"/>
<img src="figuras/silhouette_analysis06.png" width="450"/>


O coeficiente de silhueta está disponível como ``silhouette_samples`` no módulo métrico do scikit-learn e, opcionalmente, o ``silhouette_scores`` pode ser importado.

http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

### Vantagens e Desvantagens do K-Means
**Vantagens**
- Utiliza princípios simples que podem ser explicados em termos não-estatísticos.
- Altamente flexível e pode ser adaptado com ajustes simples para resolver quase todas as suas deficiências.
- Funciona muito bem em muitos casos de uso do mundo real.

**Desvantagens**
- Não é tão sofisticado quanto os algoritmos de clustering mais modernos.
- Como ele utiliza um elemento de chance aleatória, não é garantido encontrar o conjunto ideal de clusters.
- Requer uma estimativa razoável de quantos clusters existem naturalmente nos dados.
- Os clusters não se sobrepõem e não são hierárquicos